In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


#Read the dataset

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline


In [ ]:
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)
liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

# Remove duplicates based on specific columns (e.g., 'Comments' and 'TDType')
liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])

liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

# Replace values with 'WITHOUT CLASSIFICATION'
values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


##LR couunt

In [ ]:
data = liu_

X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
# Further split the training set into training and validation sets (80% of training goes to training, 20% goes to validation)
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


pipeline = Pipeline([
    ('COUNT', CountVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(random_state=42, class_weight='balanced'))
])

# hyperparameters grid
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],  # Regularisation strength
    'clf__penalty': ['l2', 'elasticnet'],  # Regularisation penalty type
    'clf__max_iter': [100, 200, 500]  # Number of iterations for convergence
}


grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_final, y_train_final)

# Print the best parameters found by GridSearchCV
print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("\nValidation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters found: {'clf__C': 10, 'clf__max_iter': 100, 'clf__penalty': 'l2'}

Validation Confusion Matrix:
[[  60    1    2   53    0    4    0   23]
 [   2   36    2   15    0    4    0    8]
 [   2    5   55   20    1    5    4    7]
 [  52   21   51 1357    0  110   16  109]
 [   2    0    0    4   11    1    1    1]
 [   2    1    6   76    3  242    4   14]
 [   1    2    2   13    0    2   79    6]
 [  20    9    5   84    0    8    6 3590]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.43      0.42      0.42       143
         COMPATIBILITY       0.48      0.54      0.51        67
                DEFECT       0.45      0.56      0.50        99
                DESIGN       0.84      0.79      0.81      1716
         DOCUMENTATION       0.73      0.55      0.63        20
        IMPLEMENTATION       0.64      0.70      0.67       3